In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import random

In [3]:
N = 33
R = 3
K = 500

In [4]:
#重力モデルで推移確率行列を作成 
def getGravity(distance, popularity): #distanceは距離行列、popularityはクラス分の人気度
  C = 0.1475
  alpha = 1.0
  beta = 1.0
  eta = 0.5
  class_number = len(popularity[0]) #クラス数
  tp = np.zeros((len(distance) * class_number, len(distance) * class_number))
  for r in range(class_number):
    for i in range(len(distance) * r, len(distance) * (r+1)):
      for j in range(len(distance) * r, len(distance) * (r+1)):
        if distance[i % len(distance)][j % len(distance)] > 0:
          tp[i][j] = C * (popularity[i % len(distance)][r]**alpha) * (popularity[j % len(distance)][r]**beta) / (distance[i % len(distance)][j % len(distance)]**eta)
  row_sum = np.sum(tp, axis=1) #行和を算出
  for i in range(len(tp)): #行和を1にする
    if row_sum[i] > 0:
      tp[i] /= row_sum[i]
  return tp 

In [5]:
def getTransitionProbability(N, R):
  #(1) 拠点の設置と拠点間距離
  node_position_x = [random.randint(0,500) for i in range(N)]
  node_position_y = [random.randint(0,500) for i in range(N)]
  from_id = [] #DF作成用
  to_id = [] #DF作成用
  distance = []
  for i in range(N):
    for j in range(i+1,N):
      from_id.append(i)
      to_id.append(j)
      distance.append(np.sqrt((node_position_x[i]-node_position_x[j])**2 + (node_position_y[i]-node_position_y[j])**2))
  df_distance = pd.DataFrame({ 'from_id' : from_id,
                          'to_id' : to_id,
                          'distance' : distance
  })#データフレーム化
  #距離行列の作成
  distance_matrix = np.zeros((N,N))
  for row in df_distance.itertuples(): #右三角行列で作成される
    distance_matrix[int(row.from_id)][int(row.to_id)] = row.distance
  for i in range(len(distance_matrix)): #下三角に値を入れる(対象)
    for j in range(i+1, len(distance_matrix)):
      distance_matrix[j][i] = distance_matrix[i][j]

  #(2)人気度の設定
  popularity = np.abs(np.random.normal(10, 2, (N, R)))

  #(3)推移確率行列の作成
  tp = getGravity(distance_matrix, popularity)

  #(4)拠点情報(拠点番号、位置(x,y)、人気度(クラス数分))の生成
  df_node = pd.DataFrame({ 'node_number' : range(N),
                        'position_x' : node_position_x,
                        'position_y' : node_position_y,
  })
  df_node.set_index('node_number',inplace=True)
  #popularityを追加
  columns = ['popurarity_'+str(i) for i in range(R)]
  for i, val in enumerate(columns):
    df_node[val] = popularity[:, i]

  #(5)情報の保存
  df_node.to_csv('/content/drive/MyDrive/研究/BCMP/TransitionProbability/csv/node_N'+str(N)+'_R'+str(R)+'_K'+str(K)+'.csv', index=True)
  df_distance.to_csv('/content/drive/MyDrive/研究/BCMP/TransitionProbability/csv/distance_N'+str(N)+'_R'+str(R)+'_K'+str(K)+'.csv', index=True)
  pd.DataFrame(distance_matrix).to_csv('/content/drive/MyDrive/研究/BCMP/TransitionProbability/csv/distance_matrix_N'+str(N)+'_R'+str(R)+'_K'+str(K)+'.csv', index=True)
  pd.DataFrame(tp).to_csv('/content/drive/MyDrive/研究/BCMP/TransitionProbability/csv/transition_probability_N'+str(N)+'_R'+str(R)+'_K'+str(K)+'.csv', index=True)

In [6]:
getTransitionProbability(N,R)